# Monocular Depth Estimation

Based on https://github.com/compphoto/BoostingMonocularDepth

Made by [Artem Konevskikh](https://aiculedssul.net/)

In [ ]:
#@title Install
#@markdown This cell will download all the needed files
# Clone git repo
!git clone https://github.com/compphoto/BoostingMonocularDepth.git

!wget https://sfu.ca/~yagiz/CVPR21/latest_net_G.pth
# !gdown https://drive.google.com/u/0/uc?id=1cU2y-kMbt0Sf00Ns4CN2oO9qPJ8BensP&export=download

# Downloading merge model weights
!mkdir -p /content/BoostingMonocularDepth/pix2pix/checkpoints/mergemodel/
!mv latest_net_G.pth /content/BoostingMonocularDepth/pix2pix/checkpoints/mergemodel/

# Downloading Midas weights
!wget https://github.com/AlexeyAB/MiDaS/releases/download/midas_dpt/midas_v21-f6b98070.pt
!mv midas_v21-f6b98070.pt /content/BoostingMonocularDepth/midas/model.pt

# # Downloading LeRes weights
!wget https://cloudstor.aarnet.edu.au/plus/s/lTIJF4vrvHCAI31/download
!mv download /content/BoostingMonocularDepth/res101.pth

# Create folders
!mkdir /content/images /content/depthmaps /content/results

In [11]:
#@title Load libraries and functions
import glob
import shlex
import os
import struct
import numpy as np
import cv2
from tqdm.notebook import tqdm

def pointcloud(img, depth, extrude=1):
    w, h = depth.shape
    
    depth = extrude*depth/255

    pixel_x, pixel_y = np.meshgrid(np.linspace(0, h - 1, h),np.linspace(0, w - 1, w))
    camera_points = np.zeros((np.size(pixel_x), 3))
    camera_points[:, 0] = np.reshape(pixel_x, -1)
    camera_points[:, 1] = np.reshape(pixel_y, -1)
    camera_points[:, 2] = np.reshape(depth, -1)

    color_points = img.reshape(-1, 3)

    valid_depth_ind = np.where(depth.flatten() > 0)[0]
    camera_points = camera_points[valid_depth_ind, :]
    color_points = color_points[valid_depth_ind, :]
    color_points = color_points.astype(int)

    return camera_points, color_points


def write_pointcloud(filename, xyz_points, rgb_points=None):
    """ creates a .ply file of the generated point clouds 
    """

    assert xyz_points.shape[1] == 3, 'Input XYZ points should be Nx3 float array'
    if rgb_points is None:
        rgb_points = np.ones(xyz_points.shape).astype(np.uint8) * 255
    assert xyz_points.shape == rgb_points.shape, 'Input RGB colors should be Nx3 float array and have same size as input XYZ points'

    # Write header of .ply file
    with open(filename, 'wb') as fid:
        fid.write(bytes('ply\n', 'utf-8'))
        fid.write(bytes('format binary_little_endian 1.0\n', 'utf-8'))
        fid.write(bytes(f'element vertex {xyz_points.shape[0]}\n', 'utf-8'))
        fid.write(bytes('property float x\n', 'utf-8'))
        fid.write(bytes('property float y\n', 'utf-8'))
        fid.write(bytes('property float z\n', 'utf-8'))
        fid.write(bytes('property uchar red\n', 'utf-8'))
        fid.write(bytes('property uchar green\n', 'utf-8'))
        fid.write(bytes('property uchar blue\n', 'utf-8'))
        fid.write(bytes('end_header\n', 'utf-8'))

        # Write 3D points to .ply file
        for i in range(xyz_points.shape[0]):
            fid.write(bytearray(struct.pack("fffBBB", xyz_points[i, 0], xyz_points[i, 1], xyz_points[i, 2],
                                            rgb_points[i, 0], rgb_points[i, 1],
                                            rgb_points[i, 2])))

In [ ]:
%cd BoostingMonocularDepth

#@title Create Depth Maps
#@markdown Input images
input_dir = '/content/images' #@param {type:"string"}
#@markdown Output directory
output_dir = '/content/depthmaps' #@param {type:"string"}
#@markdown Select the network model to reconstruct depth map. `LeRes` gives beter resolution, while `MiDas` works faster
depth_net = "LeRes" #@param ["LeRes","MiDas"]

model = 2 if depth_net == "LeRes" else 0

input_dir = shlex.quote(input_dir)
output_dir = shlex.quote(output_dir)
if not os.path.isdir(output_dir):
  !mkdir $output_dir 

!python run.py --Final --data_dir $input_dir --output_dir $output_dir --depthNet $model




In [ ]:
#@title Depth Maps to Point Clouds
#@markdown Images folder
input_dir = '/content/images' #@param {type:"string"}
#@markdown Depth Maps folder
depthmap_dir = '/content/depthmaps/' #@param {type:"string"}
#@markdown Output Point Clouds directory
output_dir = '/content/results/' #@param {type:"string"}
#@markdown Extrusion coefficient
extrude=300 #@param {type:"number"}
#@markdown Invert texture
invert=False #@param {type: "boolean"}

output_dir = shlex.quote(output_dir)
if not os.path.isdir(output_dir):
  !mkdir $output_dir 

for f in glob.glob(f'{input_dir}/*.*'):
  print(f)
  model_name = f.split('/')[-1].split('.')[0]
  texture_name = f.split('/')[-1]
  model = f'{output_dir}/{model_name}.ply'
  depth_map = f'{depthmap_dir}/{model_name}.png'

  image = cv2.imread(f, 1)
  depthmap = cv2.imread(depth_map, 1)
  depthmap = np.dot(depthmap[...,:3], [0.299, 0.587, 0.114])
  camera_points, color_points = pointcloud(image, depthmap, extrude=extrude)
  if invert:
    color_points = 255 - color_points
  write_pointcloud(model, camera_points, color_points)
